# Use Case Description

Automate security monitoring and guidance during the software development lifecycle (SDLC) by embedding a GitHub Action that triggers on pull requests (PRs). The system summarizes changes, evaluates them for security risks, and provides actionable recommendations to reviewers. This turns every PR into an opportunity for proactive security posture improvement — not just static scanning, but contextual reasoning.

## Model used for this use case
Both Instruct Model and Reasoning Model would be suitable for this task. In this example, we used Instruct Model.

## SetUp

The setup scripts below are essentially the same as those in the [Quickstart (Instruct Model)](https://github.com/RobustIntelligence/foundation-ai-cookbook/blob/main/1_quickstarts/Preview_Quickstart_instruct_model.ipynb)|

In [1]:
import os
import transformers
import torch
from IPython.display import display, Markdown

HF_TOKEN = os.getenv("HF_TOKEN")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
MODEL_ID = "" # To be relaced with the final model name

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16, # this model's tensor_type is BF16
    token=HF_TOKEN,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
generation_args = {
    "max_new_tokens": 1024,
    "temperature": None,
    "repetition_penalty": 1.2,
    "do_sample": False,
    "use_cache": True,
    "eos_token_id": tokenizer.eos_token_id,
    "pad_token_id": tokenizer.pad_token_id,
}

In [4]:
import re

def inference(prompt, system_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]
    inputs = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    
    inputs = tokenizer(inputs, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            **generation_args,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens = False)
    
    # extract the assistant response part only
    match = re.search(r"<\|assistant\|>(.*?)<\|end_of_text\|>", response, re.DOTALL)
    
    return match.group(1).strip()

## Security Analysis Utils

In this sample PR Diff, Replaced use of `eval()` with `ast.literal_eval()` — this is a significant security improvement. `eval()` can execute arbitrary code, whereas `ast.literal_eval()` safely evaluates strings with Python literals.

In [5]:
pr_diff_text = """
diff --git a/app/utils.py b/app/utils.py
index e69de29..b25a3c0 100644
--- a/app/utils.py
+++ b/app/utils.py
@@ def process_input(data):
-    return eval(data)
+    import ast
+    return ast.literal_eval(data)

diff --git a/.github/workflows/deploy.yml b/.github/workflows/deploy.yml
index a1b2c3d..d4e5f6g 100644
--- a/.github/workflows/deploy.yml
+++ b/.github/workflows/deploy.yml
@@ steps:
-      run: npm run deploy
+      run: |
+        set -e
+        npm run lint
+        npm test
+        npm run deploy
"""


In [6]:
SYSTEM_PROMPT = "You are a security expert reviewing changes introduced in a pull request."

prompt = f'''Analyze the following code diff and identify:
1. Security-relevant changes
2. Any potential vulnerabilities introduced
3. A clear summary of affected areas
4. Recommended remediations (if needed)

## PULL REQUEST DIFF
{pr_diff_text}'''

In [7]:
response = inference(prompt, SYSTEM_PROMPT)
display(Markdown(response))

### Analysis:

#### 1. **Security-Relevant Changes**

**In `app/utils.py`:**
The change from using Python's built-in `eval` function to use `ast.literal_eval` for processing input data is significant.

*Before:* 
```python
def process_input(data):
    return eval(data) # This can evaluate arbitrary expressions, which poses serious security risks if 'data' contains untrusted content.
```

*After:*
```python
import ast
def process_input(data):
    return ast.literal_eval(data) # Safer than eval as it only evaluates strings that could be assigned to an identifier or expression
```
Using `eval()` on user-provided data has been replaced with `ast.literal_eval()`, which is generally safer because it does not allow execution of arbitrary commands but rather converts a string into a python literal object like int, float, str etc., without evaluating any expressions.

However, this doesn't completely eliminate all risk since `literal_eval` still allows evaluation of certain constructs such as numbers, strings, lists, tuples, sets, dictionaries, booleans, None, and identifiers that resolve to constants.

**In `.github/workflows/deploy.yml`:**
There have been no direct security-related changes here; however, there might be indirect implications due to improved testing (`npm test`) being added before deployment.

*Before:*
```yaml
steps:
  - name: Build and Deploy
    run: npm run deploy
```

*After:*
```yaml
steps:
  - name: Lint & Test Before Deployment
    run: |
      set -e
      npm run lint
      npm test
      npm run deploy
```
Adding linter checks and tests before deploying ensures better quality control and potentially catches issues earlier, indirectly contributing to overall system stability and reducing the likelihood of introducing new bugs or regressions that may lead to security problems.

#### 2. **Potential Vulnerabilities Introduced**

While the switch from `eval` to `ast.literal_eval` mitigates some risks associated with executing arbitrary code, it’s important to note that `ast.literal_eval` isn’t entirely safe against all types of attacks. For instance, it will convert a string representing a list comprehension or dictionary comprehension into a real Python object, which could execute malicious logic under specific circumstances.

Moreover, while the workflow file itself hasn't changed directly related to security concerns, relying solely on manual review processes within CI/CD pipelines can sometimes miss critical errors or misconfigurations leading to other kinds of pipeline failures or even leaks.

#### 3. **Summary of Affected Areas**

- The primary focus of these changes is on improving safety when handling user-supplied data by replacing `eval` with `ast.literal_eval`.
- There was also an enhancement made to the build and release process by adding static analysis via `npm run lint` and unit tests through `npm test` prior to running the actual deployment command.

#### 4. **Recommended Remediations (If Needed)**

Given the context provided:

- It would be advisable to conduct thorough testing around how complex inputs are handled after switching to `ast.literal_eval`. Ensure that edge cases involving nested structures (like deeply-nested lists/dicts containing various types including possibly unsafe ones) do not pose unexpected risks.
  
- While `ast.literal_eval` improves upon `eval`, consider further hardening by validating and sanitizing input where possible instead of relying purely on parsing. 

- Continuously monitor the effectiveness of the newly included pre-deployment checks (`lint` and `test`). If they uncover frequent issues, reconsider whether more robust automated checks should be integrated into the pipeline.

- Regularly audit third-party dependencies used during both development and production stages to ensure no known vulnerabilities exist in them.

By making these considerations, you'll help maintain the integrity and security posture of your application over time.